In [1]:
import tensorflow as tf
import json
import numpy as np
from sklearn.model_selection import train_test_split

2023-06-04 05:31:28.100262: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-04 05:31:28.137155: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-04 05:31:28.137745: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-04 05:31:29.185383: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
with open("../dataset.json") as f:
    data = json.load(f)

X = []
Y = []
Y_flat = []

for i in data:
    X.append(np.array(i['vector']))    
    Y.append(np.array(i['matrix']))
    
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)
Y_train_flat =[i.flatten() for i in Y_train]
Y_test_flat =[i.flatten() for i in Y_test]

In [3]:
# Define the neural network architecture
def build_crazy_model(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(2048, activation='relu'),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(output_dim)
    ])
    return model
def build_model(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(196)  # Output layer with shape (196,)
    ])
    return model

def build_model_recursive(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Reshape((1, 256)),  # Reshape input to match LSTM requirements
        # tf.keras.layers.LSTM(512, return_sequences=True),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256, activation='relu')),
        tf.keras.layers.LSTM(512),
        tf.keras.layers.Dense(output_dim)
    ])
    return model

def build_model_convolutional(input_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(7*7*128, activation='relu'),
        tf.keras.layers.Reshape((7, 7, 128)),
        tf.keras.layers.Conv2DTranspose(32, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
        tf.keras.layers.Conv2DTranspose(2, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
        tf.keras.layers.Conv2D(1, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
        tf.keras.layers.Reshape((14, 14, 1))  # Update the output shape to (14, 14, 1)
    ])
    return model

In [4]:
# Define the input and output dimensions
input_dim = 128  # Dimensionality of the input vector
output_dim = 196  # Dimensionality of the output vector

In [5]:
# Build the model
model = build_model_recursive(input_dim, output_dim)
model_ann = build_model(input_dim, output_dim)
model_conv = build_model_convolutional(input_dim)

# Compile the model
model.compile(optimizer='adam', loss='mse')
model_ann.compile(optimizer='adam', loss='mse')
model_conv.compile(optimizer='adam', loss='mse')

2023-06-04 05:31:30.680648: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-04 05:31:30.699117: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-06-04 05:31:31.107351: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gra

In [6]:
model_ann.fit(x=np.array(X_train),y=np.array(Y_train_flat), epochs=800)

Epoch 1/800
1/1 [==============================] - 1s 721ms/step - loss: 386264448.0000
Epoch 2/800
1/1 [==============================] - 0s 9ms/step - loss: 67166096.0000
Epoch 3/800
1/1 [==============================] - 0s 9ms/step - loss: 51381936.0000
Epoch 4/800
1/1 [==============================] - 0s 10ms/step - loss: 51938412.0000
Epoch 5/800
1/1 [==============================] - 0s 10ms/step - loss: 36050080.0000
Epoch 6/800
1/1 [==============================] - 0s 9ms/step - loss: 20752550.0000
Epoch 7/800
1/1 [==============================] - 0s 9ms/step - loss: 11717197.0000
Epoch 8/800
1/1 [==============================] - 0s 9ms/step - loss: 8115269.5000
Epoch 9/800
1/1 [==============================] - 0s 10ms/step - loss: 6703776.0000
Epoch 10/800
1/1 [==============================] - 0s 9ms/step - loss: 5532348.5000
Epoch 11/800
1/1 [==============================] - 0s 9ms/step - loss: 4178526.5000
Epoch 12/800
1/1 [==============================] - 0s 9ms/st

1/1 [==============================] - 0s 8ms/step - loss: 80.5280
Epoch 100/800
1/1 [==============================] - 0s 9ms/step - loss: 78.8119
Epoch 101/800
1/1 [==============================] - 0s 9ms/step - loss: 80.6140
Epoch 102/800
1/1 [==============================] - 0s 10ms/step - loss: 78.2030
Epoch 103/800
1/1 [==============================] - 0s 9ms/step - loss: 71.1855
Epoch 104/800
1/1 [==============================] - 0s 8ms/step - loss: 63.6042
Epoch 105/800
1/1 [==============================] - 0s 8ms/step - loss: 56.0878
Epoch 106/800
1/1 [==============================] - 0s 9ms/step - loss: 49.7788
Epoch 107/800
1/1 [==============================] - 0s 9ms/step - loss: 45.8595
Epoch 108/800
1/1 [==============================] - 0s 10ms/step - loss: 44.3463
Epoch 109/800
1/1 [==============================] - 0s 9ms/step - loss: 43.3716
Epoch 110/800
1/1 [==============================] - 0s 8ms/step - loss: 41.6767
Epoch 111/800
1/1 [=====================

1/1 [==============================] - 0s 9ms/step - loss: 20.8403
Epoch 201/800
1/1 [==============================] - 0s 9ms/step - loss: 20.8354
Epoch 202/800
1/1 [==============================] - 0s 9ms/step - loss: 20.8302
Epoch 203/800
1/1 [==============================] - 0s 9ms/step - loss: 20.8246
Epoch 204/800
1/1 [==============================] - 0s 9ms/step - loss: 20.8198
Epoch 205/800
1/1 [==============================] - 0s 9ms/step - loss: 20.8148
Epoch 206/800
1/1 [==============================] - 0s 9ms/step - loss: 20.8098
Epoch 207/800
1/1 [==============================] - 0s 9ms/step - loss: 20.8050
Epoch 208/800
1/1 [==============================] - 0s 9ms/step - loss: 20.7999
Epoch 209/800
1/1 [==============================] - 0s 8ms/step - loss: 20.7947
Epoch 210/800
1/1 [==============================] - 0s 9ms/step - loss: 20.7899
Epoch 211/800
1/1 [==============================] - 0s 8ms/step - loss: 20.7850
Epoch 212/800
1/1 [=======================

1/1 [==============================] - 0s 9ms/step - loss: 20.3607
Epoch 302/800
1/1 [==============================] - 0s 9ms/step - loss: 20.3560
Epoch 303/800
1/1 [==============================] - 0s 9ms/step - loss: 20.3512
Epoch 304/800
1/1 [==============================] - 0s 9ms/step - loss: 20.3460
Epoch 305/800
1/1 [==============================] - 0s 8ms/step - loss: 20.3412
Epoch 306/800
1/1 [==============================] - 0s 9ms/step - loss: 20.3363
Epoch 307/800
1/1 [==============================] - 0s 9ms/step - loss: 20.3315
Epoch 308/800
1/1 [==============================] - 0s 9ms/step - loss: 20.3269
Epoch 309/800
1/1 [==============================] - 0s 10ms/step - loss: 20.3216
Epoch 310/800
1/1 [==============================] - 0s 8ms/step - loss: 20.3168
Epoch 311/800
1/1 [==============================] - 0s 9ms/step - loss: 20.3118
Epoch 312/800
1/1 [==============================] - 0s 9ms/step - loss: 20.3067
Epoch 313/800
1/1 [======================

1/1 [==============================] - 0s 10ms/step - loss: 19.8348
Epoch 403/800
1/1 [==============================] - 0s 9ms/step - loss: 19.8292
Epoch 404/800
1/1 [==============================] - 0s 9ms/step - loss: 19.8238
Epoch 405/800
1/1 [==============================] - 0s 9ms/step - loss: 19.8180
Epoch 406/800
1/1 [==============================] - 0s 10ms/step - loss: 19.8129
Epoch 407/800
1/1 [==============================] - 0s 9ms/step - loss: 19.8074
Epoch 408/800
1/1 [==============================] - 0s 10ms/step - loss: 19.8015
Epoch 409/800
1/1 [==============================] - 0s 10ms/step - loss: 19.7961
Epoch 410/800
1/1 [==============================] - 0s 9ms/step - loss: 19.7904
Epoch 411/800
1/1 [==============================] - 0s 9ms/step - loss: 19.7850
Epoch 412/800
1/1 [==============================] - 0s 10ms/step - loss: 19.7792
Epoch 413/800
1/1 [==============================] - 0s 10ms/step - loss: 19.7737
Epoch 414/800
1/1 [=================

Epoch 503/800
1/1 [==============================] - 0s 9ms/step - loss: 34.0537
Epoch 504/800
1/1 [==============================] - 0s 9ms/step - loss: 22.2155
Epoch 505/800
1/1 [==============================] - 0s 8ms/step - loss: 52.4142
Epoch 506/800
1/1 [==============================] - 0s 9ms/step - loss: 56.5904
Epoch 507/800
1/1 [==============================] - 0s 9ms/step - loss: 27.8839
Epoch 508/800
1/1 [==============================] - 0s 9ms/step - loss: 22.0459
Epoch 509/800
1/1 [==============================] - 0s 8ms/step - loss: 42.2811
Epoch 510/800
1/1 [==============================] - 0s 9ms/step - loss: 42.7048
Epoch 511/800
1/1 [==============================] - 0s 9ms/step - loss: 23.2761
Epoch 512/800
1/1 [==============================] - 0s 9ms/step - loss: 22.1938
Epoch 513/800
1/1 [==============================] - 0s 9ms/step - loss: 35.8191
Epoch 514/800
1/1 [==============================] - 0s 9ms/step - loss: 33.4254
Epoch 515/800
1/1 [=========

1/1 [==============================] - 0s 9ms/step - loss: 18.6193
Epoch 604/800
1/1 [==============================] - 0s 9ms/step - loss: 18.6118
Epoch 605/800
1/1 [==============================] - 0s 9ms/step - loss: 18.6045
Epoch 606/800
1/1 [==============================] - 0s 8ms/step - loss: 18.5981
Epoch 607/800
1/1 [==============================] - 0s 8ms/step - loss: 18.5920
Epoch 608/800
1/1 [==============================] - 0s 9ms/step - loss: 18.5861
Epoch 609/800
1/1 [==============================] - 0s 9ms/step - loss: 18.5794
Epoch 610/800
1/1 [==============================] - 0s 9ms/step - loss: 18.5724
Epoch 611/800
1/1 [==============================] - 0s 9ms/step - loss: 18.5656
Epoch 612/800
1/1 [==============================] - 0s 9ms/step - loss: 18.5591
Epoch 613/800
1/1 [==============================] - 0s 9ms/step - loss: 18.5530
Epoch 614/800
1/1 [==============================] - 0s 10ms/step - loss: 18.5469
Epoch 615/800
1/1 [======================

1/1 [==============================] - 0s 10ms/step - loss: 17.9462
Epoch 705/800
1/1 [==============================] - 0s 9ms/step - loss: 17.9403
Epoch 706/800
1/1 [==============================] - 0s 9ms/step - loss: 17.9340
Epoch 707/800
1/1 [==============================] - 0s 9ms/step - loss: 17.9287
Epoch 708/800
1/1 [==============================] - 0s 9ms/step - loss: 17.9247
Epoch 709/800
1/1 [==============================] - 0s 9ms/step - loss: 17.9224
Epoch 710/800
1/1 [==============================] - 0s 8ms/step - loss: 17.9241
Epoch 711/800
1/1 [==============================] - 0s 10ms/step - loss: 17.9336
Epoch 712/800
1/1 [==============================] - 0s 9ms/step - loss: 17.9581
Epoch 713/800
1/1 [==============================] - 0s 10ms/step - loss: 18.0108
Epoch 714/800
1/1 [==============================] - 0s 9ms/step - loss: 18.1203
Epoch 715/800
1/1 [==============================] - 0s 9ms/step - loss: 18.3402
Epoch 716/800
1/1 [====================

In [7]:
model_conv.fit(x=np.array(X_train),y=np.array(Y_train), epochs=100)

Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 152434.6406
Epoch 2/100
1/1 [==============================] - 0s 19ms/step - loss: 121912.4688
Epoch 3/100
1/1 [==============================] - 0s 27ms/step - loss: 6011.2339
Epoch 4/100
1/1 [==============================] - 0s 24ms/step - loss: 0.0294
Epoch 5/100
1/1 [==============================] - 0s 23ms/step - loss: 0.0294
Epoch 6/100
1/1 [==============================] - 0s 18ms/step - loss: 0.0294
Epoch 7/100
1/1 [==============================] - 0s 19ms/step - loss: 0.0294
Epoch 8/100
1/1 [==============================] - 0s 18ms/step - loss: 0.0294
Epoch 9/100
1/1 [==============================] - 0s 18ms/step - loss: 0.0294
Epoch 10/100
1/1 [==============================] - 0s 23ms/step - loss: 0.0294
Epoch 11/100
1/1 [==============================] - 0s 20ms/step - loss: 0.0294
Epoch 12/100
1/1 [==============================] - 0s 24ms/step - loss: 0.0294
Epoch 13/100
1/1 [====================

In [8]:
model.fit(x=np.array(X_train),y=np.array(Y_train_flat), epochs=100)

Epoch 1/100


2023-06-04 05:31:44.104195: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-04 05:31:44.106500: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-04 05:31:44.108168: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 3s 3s/step - loss: 0.2782
Epoch 2/100
1/1 [==============================] - 0s 18ms/step - loss: 0.2122
Epoch 3/100
1/1 [==============================] - 0s 17ms/step - loss: 0.1592
Epoch 4/100
1/1 [==============================] - 0s 17ms/step - loss: 0.1350
Epoch 5/100
1/1 [==============================] - 0s 19ms/step - loss: 0.1084
Epoch 6/100
1/1 [==============================] - 0s 17ms/step - loss: 0.1027
Epoch 7/100
1/1 [==============================] - 0s 18ms/step - loss: 0.0982
Epoch 8/100
1/1 [==============================] - 0s 17ms/step - loss: 0.0987
Epoch 9/100
1/1 [==============================] - 0s 17ms/step - loss: 0.0868
Epoch 10/100
1/1 [==============================] - 0s 17ms/step - loss: 0.0923
Epoch 11/100
1/1 [==============================] - 0s 17ms/step - loss: 0.0734
Epoch 12/100
1/1 [==============================] - 0s 17ms/step - loss: 0.0675
Epoch 13/100
1/1 [==============================] - 0s 17ms/ste

In [9]:
Y_predicted = model.predict(np.array(X_test))
Y_ann_predicted = model_ann.predict(np.array(X_test))
Y_conv_predicted = model_conv.predict(np.array(X_test))

1/1 [==============================] - 0s 491ms/step


2023-06-04 05:31:48.704987: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-04 05:31:48.707206: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-04 05:31:48.709389: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 0s 100ms/step


In [10]:
Y_predicted = [np.reshape(i, (14,14)) for i in Y_predicted]
Y_predicted

[array([[ 0.00347963, -0.00314793,  0.00541394,  0.00701526,  0.00879856,
          0.01818691,  0.00343318,  0.00462492,  0.0098026 ,  0.00250118,
          0.01049848,  0.01032894,  0.014435  ,  0.03179706],
        [ 0.00727427,  0.01222352,  0.00644533,  0.01049972,  0.01633083,
          0.02689863,  0.01550565,  0.01138067,  0.01837157,  0.01010589,
          0.02012957,  0.01441736,  0.00818152,  0.04593824],
        [ 0.00408015,  0.00062616,  0.00381978,  0.01263655,  0.01009861,
          0.02398376,  0.00725157,  0.01377535,  0.01273992,  0.00898343,
          0.00401457,  0.01461365,  0.01188799,  0.03747535],
        [ 0.0055582 ,  0.00660851,  0.00755708,  0.03177254,  0.02393388,
          0.04203806,  0.01580545,  0.01639135,  0.0260298 ,  0.01501617,
          0.02614348,  0.02414088,  0.02017305,  0.0697322 ],
        [ 0.0086903 ,  0.01770846,  0.00790202,  0.01354122,  0.04466427,
          0.06908846,  0.03453852,  0.02267153,  0.04225882,  0.02739954,
          0.

In [11]:
Y_ann_predicted = [np.reshape(i, (14,14)) for i in Y_ann_predicted]
Y_ann_predicted

[array([[ -2.1802926 ,   6.0974483 ,  -2.356477  ,   7.4431777 ,
          -3.2943525 ,   2.5987904 ,   1.5592139 ,  13.008511  ,
          -6.811128  ,   3.5930233 ,  -4.1034355 ,  12.128581  ,
          -0.9236075 ,  -5.9316354 ],
        [ -4.6045732 ,  -5.331223  ,   3.760382  ,   2.070983  ,
           3.2551317 ,   1.5480508 ,  -0.93795973,   9.413406  ,
         -13.30395   ,  -1.0505989 ,   7.584873  ,   2.657093  ,
          -7.2975984 ,   4.197778  ],
        [  2.043909  ,   5.8877726 ,  -5.9498916 ,   4.77129   ,
          -8.073128  , -19.24335   ,  -4.5740757 ,  -0.09522682,
          -0.9447339 ,  -3.839604  ,   6.58621   ,  12.027394  ,
           9.9736805 ,   3.1435115 ],
        [-17.352478  ,  -6.0366826 ,  -4.740215  , -15.041119  ,
          -2.2660632 ,   6.564938  , -10.845716  ,  -4.1607366 ,
           6.5366306 ,   1.119189  ,  18.657623  ,   8.020272  ,
           2.375167  ,  -2.7087348 ],
        [ -2.554228  ,  -4.4926133 ,  -3.5885372 ,  -0.8541229 ,
   

In [12]:
Y_conv_predicted

array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
 